두번째 셀의 3번줄이 수집시간인데 720분으로 설정되어있어서 매일 아침11시에 실행시키시고, 오후 11시에 코드가 종료될겁니다

# 셀 실행전 매일 수정해주셔야 될것 하나:
# 두번째 셀에서 밑에서 두번째 줄에 저장파일명을 바꿔줘야 되는데 7000_요일.csv로 바꿔 주심 됩니다~

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
import time
import sklearn.preprocessing

pd.set_option("display.max_columns", None)
warnings.filterwarnings(action='ignore')

serviceKey='bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ=='
p5100 ={'serviceKey' :serviceKey,
             'routeId':'200000115','stationId':'228000703'}#5100 외대
p7000 ={'serviceKey' :serviceKey,
             'routeId':'200000112','stationId':'228000703'}
class Bus():
    def __init__(self,params):
        self.params=params
        self.bus_time=[]
        
    def repet(self):
        url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
        response = requests.get(url, params=self.params)
        soup=BeautifulSoup(response.text,'xml')
        return soup

        
    def current_buses(self): 
        url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
        mod_params ={'serviceKey' : self.params['serviceKey'],
                  'routeId':self.params['routeId']}

        response=requests.get(url,params=mod_params)
        soup=BeautifulSoup(response.text,'xml')
        locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
        return locations


def trim_data(df,FileName):
    df['시간'] = df['시간'].apply(pd.to_datetime)
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['second']=df['시간'].dt.second
    df['day'] = df['시간'].dt.weekday 

    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
    
    batch=df.index[df.arrived==1]
    df=df.iloc[:batch[-1]+1,:]
    #7/12 -----------------------------------
    batch=df.index[df.arrived==1]
    temp=0
    for x in batch:
        for y in range(temp,x): 
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0
    # 7/12 -----------------------------------

    df = df.set_index('시간')
    df.to_csv(f'./{FileName}')        # 동일 폴더에 파일이 저장될 거임
    return df

In [9]:
past=4
pBus=Bus(p7000)
for x in range(720): # 12시간
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
        
    except requests.Timeout as err:
        continue
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'7000_fri.csv')
mod_df

0 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0]]
1 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0]]
2 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.724', [7, 10, 15, 17, 21, 23, 30, 32, 40, 43, 45], 0]]
3 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.724', [7, 10, 15, 17, 21, 23, 30, 32, 40, 43, 45], 0], ['2022-08-11 18:12:14.901', [1, 8, 11, 15, 17, 22, 24, 30, 33, 40, 43, 45], 1]]
4 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.7

14 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.724', [7, 10, 15, 17, 21, 23, 30, 32, 40, 43, 45], 0], ['2022-08-11 18:12:14.901', [1, 8, 11, 15, 17, 22, 24, 30, 33, 40, 43, 45], 1], ['2022-08-11 18:13:15.126', [3, 8, 12, 16, 18, 22, 24, 31, 33, 41, 44, 46], 0], ['2022-08-11 18:14:18.365', [4, 8, 12, 16, 19, 23, 24, 31, 34, 41, 44, 46], 0], ['2022-08-11 18:15:18.590', [4, 9, 13, 16, 19, 23, 24, 31, 35, 42, 44, 47], 0], ['2022-08-11 18:16:18.768', [4, 9, 13, 17, 20, 23, 24, 31, 35, 43, 45, 48], 0], ['2022-08-11 18:17:19.023', [4, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 48], 0], ['2022-08-11 18:18:19.201', [5, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 49], 0], ['2022-08-11 18:19:19.395', [6, 10, 15, 18, 22, 24, 25, 32, 37, 44, 46, 49], 0], ['2022-08-11 18:20:22.791', [7, 10, 15, 19, 22, 24, 26, 32, 37, 44, 46, 50], 0], ['2022-08-11 18:21:23.078', [7, 11, 

20 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.724', [7, 10, 15, 17, 21, 23, 30, 32, 40, 43, 45], 0], ['2022-08-11 18:12:14.901', [1, 8, 11, 15, 17, 22, 24, 30, 33, 40, 43, 45], 1], ['2022-08-11 18:13:15.126', [3, 8, 12, 16, 18, 22, 24, 31, 33, 41, 44, 46], 0], ['2022-08-11 18:14:18.365', [4, 8, 12, 16, 19, 23, 24, 31, 34, 41, 44, 46], 0], ['2022-08-11 18:15:18.590', [4, 9, 13, 16, 19, 23, 24, 31, 35, 42, 44, 47], 0], ['2022-08-11 18:16:18.768', [4, 9, 13, 17, 20, 23, 24, 31, 35, 43, 45, 48], 0], ['2022-08-11 18:17:19.023', [4, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 48], 0], ['2022-08-11 18:18:19.201', [5, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 49], 0], ['2022-08-11 18:19:19.395', [6, 10, 15, 18, 22, 24, 25, 32, 37, 44, 46, 49], 0], ['2022-08-11 18:20:22.791', [7, 10, 15, 19, 22, 24, 26, 32, 37, 44, 46, 50], 0], ['2022-08-11 18:21:23.078', [7, 11, 

25 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.724', [7, 10, 15, 17, 21, 23, 30, 32, 40, 43, 45], 0], ['2022-08-11 18:12:14.901', [1, 8, 11, 15, 17, 22, 24, 30, 33, 40, 43, 45], 1], ['2022-08-11 18:13:15.126', [3, 8, 12, 16, 18, 22, 24, 31, 33, 41, 44, 46], 0], ['2022-08-11 18:14:18.365', [4, 8, 12, 16, 19, 23, 24, 31, 34, 41, 44, 46], 0], ['2022-08-11 18:15:18.590', [4, 9, 13, 16, 19, 23, 24, 31, 35, 42, 44, 47], 0], ['2022-08-11 18:16:18.768', [4, 9, 13, 17, 20, 23, 24, 31, 35, 43, 45, 48], 0], ['2022-08-11 18:17:19.023', [4, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 48], 0], ['2022-08-11 18:18:19.201', [5, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 49], 0], ['2022-08-11 18:19:19.395', [6, 10, 15, 18, 22, 24, 25, 32, 37, 44, 46, 49], 0], ['2022-08-11 18:20:22.791', [7, 10, 15, 19, 22, 24, 26, 32, 37, 44, 46, 50], 0], ['2022-08-11 18:21:23.078', [7, 11, 

29 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.724', [7, 10, 15, 17, 21, 23, 30, 32, 40, 43, 45], 0], ['2022-08-11 18:12:14.901', [1, 8, 11, 15, 17, 22, 24, 30, 33, 40, 43, 45], 1], ['2022-08-11 18:13:15.126', [3, 8, 12, 16, 18, 22, 24, 31, 33, 41, 44, 46], 0], ['2022-08-11 18:14:18.365', [4, 8, 12, 16, 19, 23, 24, 31, 34, 41, 44, 46], 0], ['2022-08-11 18:15:18.590', [4, 9, 13, 16, 19, 23, 24, 31, 35, 42, 44, 47], 0], ['2022-08-11 18:16:18.768', [4, 9, 13, 17, 20, 23, 24, 31, 35, 43, 45, 48], 0], ['2022-08-11 18:17:19.023', [4, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 48], 0], ['2022-08-11 18:18:19.201', [5, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 49], 0], ['2022-08-11 18:19:19.395', [6, 10, 15, 18, 22, 24, 25, 32, 37, 44, 46, 49], 0], ['2022-08-11 18:20:22.791', [7, 10, 15, 19, 22, 24, 26, 32, 37, 44, 46, 50], 0], ['2022-08-11 18:21:23.078', [7, 11, 

...

236 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.724', [7, 10, 15, 17, 21, 23, 30, 32, 40, 43, 45], 0], ['2022-08-11 18:12:14.901', [1, 8, 11, 15, 17, 22, 24, 30, 33, 40, 43, 45], 1], ['2022-08-11 18:13:15.126', [3, 8, 12, 16, 18, 22, 24, 31, 33, 41, 44, 46], 0], ['2022-08-11 18:14:18.365', [4, 8, 12, 16, 19, 23, 24, 31, 34, 41, 44, 46], 0], ['2022-08-11 18:15:18.590', [4, 9, 13, 16, 19, 23, 24, 31, 35, 42, 44, 47], 0], ['2022-08-11 18:16:18.768', [4, 9, 13, 17, 20, 23, 24, 31, 35, 43, 45, 48], 0], ['2022-08-11 18:17:19.023', [4, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 48], 0], ['2022-08-11 18:18:19.201', [5, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 49], 0], ['2022-08-11 18:19:19.395', [6, 10, 15, 18, 22, 24, 25, 32, 37, 44, 46, 49], 0], ['2022-08-11 18:20:22.791', [7, 10, 15, 19, 22, 24, 26, 32, 37, 44, 46, 50], 0], ['2022-08-11 18:21:23.078', [7, 11,

237 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.724', [7, 10, 15, 17, 21, 23, 30, 32, 40, 43, 45], 0], ['2022-08-11 18:12:14.901', [1, 8, 11, 15, 17, 22, 24, 30, 33, 40, 43, 45], 1], ['2022-08-11 18:13:15.126', [3, 8, 12, 16, 18, 22, 24, 31, 33, 41, 44, 46], 0], ['2022-08-11 18:14:18.365', [4, 8, 12, 16, 19, 23, 24, 31, 34, 41, 44, 46], 0], ['2022-08-11 18:15:18.590', [4, 9, 13, 16, 19, 23, 24, 31, 35, 42, 44, 47], 0], ['2022-08-11 18:16:18.768', [4, 9, 13, 17, 20, 23, 24, 31, 35, 43, 45, 48], 0], ['2022-08-11 18:17:19.023', [4, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 48], 0], ['2022-08-11 18:18:19.201', [5, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 49], 0], ['2022-08-11 18:19:19.395', [6, 10, 15, 18, 22, 24, 25, 32, 37, 44, 46, 49], 0], ['2022-08-11 18:20:22.791', [7, 10, 15, 19, 22, 24, 26, 32, 37, 44, 46, 50], 0], ['2022-08-11 18:21:23.078', [7, 11,

238 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.724', [7, 10, 15, 17, 21, 23, 30, 32, 40, 43, 45], 0], ['2022-08-11 18:12:14.901', [1, 8, 11, 15, 17, 22, 24, 30, 33, 40, 43, 45], 1], ['2022-08-11 18:13:15.126', [3, 8, 12, 16, 18, 22, 24, 31, 33, 41, 44, 46], 0], ['2022-08-11 18:14:18.365', [4, 8, 12, 16, 19, 23, 24, 31, 34, 41, 44, 46], 0], ['2022-08-11 18:15:18.590', [4, 9, 13, 16, 19, 23, 24, 31, 35, 42, 44, 47], 0], ['2022-08-11 18:16:18.768', [4, 9, 13, 17, 20, 23, 24, 31, 35, 43, 45, 48], 0], ['2022-08-11 18:17:19.023', [4, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 48], 0], ['2022-08-11 18:18:19.201', [5, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 49], 0], ['2022-08-11 18:19:19.395', [6, 10, 15, 18, 22, 24, 25, 32, 37, 44, 46, 49], 0], ['2022-08-11 18:20:22.791', [7, 10, 15, 19, 22, 24, 26, 32, 37, 44, 46, 50], 0], ['2022-08-11 18:21:23.078', [7, 11,

239 번째 시도:  [['2022-08-11 18:09:14.337', [5, 9, 14, 16, 19, 23, 29, 32, 40, 42, 44], 0], ['2022-08-11 18:10:14.546', [6, 10, 14, 16, 20, 23, 29, 32, 40, 42, 45], 0], ['2022-08-11 18:11:14.724', [7, 10, 15, 17, 21, 23, 30, 32, 40, 43, 45], 0], ['2022-08-11 18:12:14.901', [1, 8, 11, 15, 17, 22, 24, 30, 33, 40, 43, 45], 1], ['2022-08-11 18:13:15.126', [3, 8, 12, 16, 18, 22, 24, 31, 33, 41, 44, 46], 0], ['2022-08-11 18:14:18.365', [4, 8, 12, 16, 19, 23, 24, 31, 34, 41, 44, 46], 0], ['2022-08-11 18:15:18.590', [4, 9, 13, 16, 19, 23, 24, 31, 35, 42, 44, 47], 0], ['2022-08-11 18:16:18.768', [4, 9, 13, 17, 20, 23, 24, 31, 35, 43, 45, 48], 0], ['2022-08-11 18:17:19.023', [4, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 48], 0], ['2022-08-11 18:18:19.201', [5, 9, 14, 17, 21, 23, 25, 32, 36, 43, 45, 49], 0], ['2022-08-11 18:19:19.395', [6, 10, 15, 18, 22, 24, 25, 32, 37, 44, 46, 49], 0], ['2022-08-11 18:20:22.791', [7, 10, 15, 19, 22, 24, 26, 32, 37, 44, 46, 50], 0], ['2022-08-11 18:21:23.078', [7, 11,

,arrived,hour,minute,second,day,station1,station2,station3,station4,station5,station6,station7,station8,station9,station10,station11,station12,station13,station14,station15,station16,station17,station18,station19,station20,station21,station22,station23,station24,station25,station26,station27,station28,station29,station30,station31,station32,station33,station34,station35,station36,station37,station38,station39,station40,station41,station42,station43,station44,station45,station46,station47,station48,station49,station50,station51,station52,station53,station54
시간,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-08-11 18:09:14.337,180,18,9,14,3,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2022-08-11 18:10:14.546,120,18,10,14,3,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
2022-08-11 18:11:14.724,60,18,11,14,3,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2022-08-11 18:12:14.901,0,18,12,14,3,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2022-08-11 18:13:15.126,793,18,13,15,3,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-11 22:05:07.703,242,22,5,7,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
2022-08-11 22:06:07.897,181,22,6,7,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
2022-08-11 22:07:08.075,121,22,7,8,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
